In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.datasets as dataset
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import os
%matplotlib qt

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [ ]:
# 超参数
num_input = 513*151
in_channels = 1
features = 8
num_hidden = 200
num_class = 6
batch_size = 128
learning_rate = 0.005
epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# 读取数据集
data_path = "D:\\Data_Set\\pyVHR\\MovingClass\\Class2\\Data\\"
label_path = "D:\\Data_Set\\pyVHR\\MovingClass\\Class2\\Labels\\"
data_files = os.listdir(data_path)
label_files = os.listdir(label_path)
all_data = []
all_label = []
for i in range(len(data_files)):
    data = np.loadtxt(data_path + data_files[i])
    all_data.append(data)
    label = np.loadtxt(label_path + label_files[i])
    all_label.append(label)

In [ ]:
all_data = np.array(all_data)
all_label = np.array(all_label, dtype=np.int)

In [ ]:
count = len(all_label)
train = [i for i in range(count)]
test = []
for i in range(0, count, 4):
    test.append(i)
    train.remove(i)
x_train = all_data[train]
y_train = all_label[train]
x_valid = all_data[test]
y_valid = all_label[test]

In [ ]:
# 数据预处理
train_transform = transforms.Compose([transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
# 构建Dataset类训练、测试数据， 集成Dataset，复写__getitem__
class MyDataset(Dataset):
    def __init__(self, data, label, transform):
        '''
        自制数据集的Dataset
        '''
        self.data = data
        self.labels = label
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # 返回第index个数据样本
        img = self.data[index]
        label = self.labels[index]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

train_data = MyDataset(x_train, y_train, train_transform)
Train_loader =DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_data = MyDataset(x_valid, y_valid, test_transform)
Test_loader = DataLoader(dataset=test_data, batch_size=len(test_data), shuffle=False)

In [ ]:
# 搭建CNN网络
class CNN(nn.Module):
    def __init__(self, in_channel=1, feature=8, num_class=2):
        super(CNN, self).__init__()
        self.Conv1 = nn.Conv2d(in_channels=in_channel, out_channels=feature, kernel_size=3, stride=1, padding=1)     # 卷积层1： Batch * 8 * 513 * 151
        self.Relu1 = nn.ReLU()                               # 激活层
        self.Pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化1：Batch * 8 * 256 * 75

        self.Conv2 = nn.Conv2d(feature, feature, kernel_size=3, stride=1, padding=1)                                 # 卷积层2：  Batch * 8 * 256 * 75
        self.Relu2 = nn.ReLU()                               # 激活层
        self.Pool2 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化2：Batch * 8 * 128 * 37

        self.Conv3 = nn.Conv2d(feature, feature // 2, kernel_size=3, stride=1, padding=1)                            # 卷积层3：Batch * 4 * 128 * 37
        self.Relu3 = nn.ReLU()                               # 激活层
        self.Pool3 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化3：Batch * 4 * 64 * 18

        self.Conv4 = nn.Conv2d(feature//2, 1, kernel_size=3, stride=1, padding=1)                                    # 卷积层4：Batch * 1 * 64 * 18
        self.Relu4 = nn.ReLU()                               # 激活层
        self.Pool4 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化4 Batch * 1 * 32 * 9

        self.Fc = nn.Linear(32*9, num_class)               # 全连接层输出

    # 参数前向传播
    def forward(self, x):
        x = self.Conv1(x)
        x = self.Relu1(x)
        x = self.Pool1(x)
        x = self.Conv2(x)
        x = self.Relu2(x)
        x = self.Pool2(x)
        x = self.Conv3(x)
        x = self.Relu3(x)
        x = self.Pool3(x)
        x = self.Conv4(x)
        x = self.Relu4(x)
        x = self.Pool4(x)
        x = x.reshape(x.shape[0], -1)   # [Batch, , , ] --> [Batch, ]
        x = self.Fc(x)
        return x

# 初始化模型
model = CNN(in_channel=in_channels, feature=features, num_class=2).to(device)

In [ ]:
#定义损失函数和优化器
LossFuc = nn.CrossEntropyLoss()     # 该损失函数包括了softmax，因此不需要在输出层额外添加softmax激活函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 训练
loss_lst = []
epochs = 30
for epoch in range(epochs):
    loss_epoch = 0.0
    for batch_index, (images, labels) in enumerate(Train_loader):
        images = images.to(torch.float32)
        images  = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        # 计算损失
        loss = LossFuc(outputs, labels.long())
        loss_epoch += loss

        # 梯度后向传播
        optimizer.zero_grad()   # 置零
        loss.backward()         # 向后传
        optimizer.step()        # 更新

        # if batch_index % 100 == 0:
        print('[{}/{}], [{}/{}], loss={:.4f}'.format(epoch+1, epochs, batch_index, len(Train_loader), loss))
    loss_lst.append(loss_epoch / (batch_index + 1))

In [ ]:
loss_data = [i.data.cpu().item() for i in loss_lst]
loss_data

In [ ]:
plt.plot(np.arange(0, epochs), np.array(loss_data))
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
plt.ylabel("Loss", fontproperties='Times New Roman', fontsize=18)
plt.xlabel("训练轮次", fontproperties='SimSun', fontsize=18)
# plt.savefig(r"D:\OneDrive\桌面\绘图1.svg", dpi=500)

In [ ]:
with torch.no_grad():
    correct_num = 0
    total_num = 0
    for images, labels in Test_loader:
        images = images.to(torch.float32)
        images  = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        correct_num = (predictions==labels).sum()
        total_num = (predictions.size(0))
        print("测试集精度为：{}%".format(correct_num/total_num*100))

In [ ]:
labels

In [ ]:
t_count = 0
f_count = 0
with torch.no_grad():
    t_f_num = 0
    f_t_num = 0
    t_t_num = 0
    f_f_num = 0
    i = 0
    for images, labels in Test_loader:
        for i in range(len(images)):
            image = images[i].to(torch.float32).to(device)
            label = labels[i]
            output = model(image)
            _, prediction = torch.max(output, 1)
            if label == 1:
                t_count += 1
                if prediction == 1:
                    t_t_num += 1
                elif prediction == 0:
                    t_f_num += 1
            if label == 0:
                f_count += 1
                if prediction == 0:
                    f_f_num += 1
                elif prediction == 1:
                    f_t_num += 1
        print(len(images))

In [ ]:
test_count = len(images)
print(test_count, t_count, t_f_num)
print(test_count, t_count, t_t_num)
print(test_count, f_count, f_t_num)
print(test_count, f_count, f_f_num)

In [ ]:
confun_matrix = np. array([[t_t_num, t_f_num], [f_t_num, f_f_num]])
# confun_matrix = np. array([[t_t_num / t_count, t_f_num / t_count], [f_t_num / f_count, f_f_num / f_count]])
confun_matrix = np.around(confun_matrix, decimals=4)
confun_matrix

In [ ]:
plt.figure(figsize=(13,8))
# plt.text(-0.2,0, str(confun_matrix[0,0]), fontsize=22, fontdict={'color': 'white'})
# plt.text(-0.2,1, str(confun_matrix[1,0]), fontsize=22)
# plt.text(0.8,0, str(confun_matrix[0,1]), fontsize=22)
# plt.text(0.8,1, str(confun_matrix[1,1]), fontsize=22, fontdict={'color': 'white'})
plt.text(-0.1,0, str(confun_matrix[0,0]), fontsize=22, fontdict={'color': 'white'})
plt.text(0,1, str(confun_matrix[1,0]), fontsize=22)
plt.text(1,0, str(confun_matrix[0,1]), fontsize=22)
plt.text(0.9,1, str(confun_matrix[1,1]), fontsize=22)
plt.xticks([0, 1], ['周期', '非周期'], fontproperties='SimSun', fontsize=18)
plt.yticks([0, 1], ['周期', '非周期'], fontproperties='SimSun', fontsize=18)
plt.xlabel("预测值", fontproperties='SimSun', fontsize=18)
plt.ylabel("真实值", fontproperties='SimSun', fontsize=18)
plt.imshow(confun_matrix, cmap='Blues')
plt.tight_layout()
plt.colorbar()
plt.savefig(r"D:\OneDrive\桌面\网络训练混淆矩阵.svg", dpi=500)

In [ ]:
torch.save(model, 'MotionClassTwoCNN2024.pth')

In [ ]:
tt = torch.load('MotionClassTwoCNN.pth', map_location=device)
tt.eval()

In [ ]:
# pre1 = x_valid[77]
pre1 = np.loadtxt("D:\\Data_Set\\pyVHR\\MovingClass\\Data\\1205_1.txt")
print(y_valid[77])

In [ ]:
tran = transforms.ToTensor()
pre1 = tran(pre1).to(device)
pre1 = pre1.view(1,1,513,151).to(torch.float32)

In [ ]:
predict = tt(pre1)
predict

In [ ]:
print(int(predict.argmax()))